## Classy classification

Classy classification - библиотека, в которой содержатся модели для классификации текстов. 

### Установка

In [14]:
!pip install classy-classification
!python -m spacy download ru_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-07 15:18:42.381343: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=9c068f4027dd53d3217663bf767d7ea29b0d9f2ca6ed25ebc9bb053f1b23c2fa
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


### Импорты

In [84]:
from pprint import pprint
import matplotlib.pyplot as plt
import classy_classification
import seaborn as sns
import pandas as pd
import numpy as np
import spacy 
import pickle

In [47]:
with open('/content/contract_enforcement.pkl', 'rb') as fp:
  contract_enforcement = pickle.load(fp)
contract_enforcement = pd.Series(contract_enforcement)

contract_enforcement = contract_enforcement[~contract_enforcement.isna()]
contract_enforcement = contract_enforcement.values

In [48]:
with open('/content/garantee_enforcement.pkl', 'rb') as fp:
  garantee_enforcement = pickle.load(fp)
garantee_enforcement = pd.Series(garantee_enforcement)

garantee_enforcement = garantee_enforcement[~garantee_enforcement.isna()]
garantee_enforcement = garantee_enforcement.values

In [12]:
df = pd.read_json('/content/train_with_splitted_texts.json', encoding='utf-8')

In [13]:
df.head()

,id,text,label,target
0,809436509,[Извещение о проведении открытого конкурса в э...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 6593.2...
1,854885310,[ЗАЯВКИ участника запроса котировок в электрон...,обеспечение исполнения контракта,Поставщик должен предоставить обеспечение испо...
2,4382157,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%
3,184555082,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%
4,211645258,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%


### построение модели

попробуем построить модель для "обеспечение исполнения контракта"

In [88]:
data={
    'обеспечение исполнения контракта': list(set(contract_enforcement)),
    '1': ['ghbdtn', 'd']}
nlp = spacy.blank('ru')
nlp.add_pipe(
    'text_categorizer',
    config={
        'data': data,
        'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
        'device': 'gpu'
    }
)

In [69]:
query = 'Рандомный текст для теста.'
print(nlp(query)._.cats)


{'обеспечение исполнения контракта': 0.7104710019488713, '1': 0.2895289980511286}


In [75]:
query = 'Размер обеспечения исполнения Контракта составляет 5 (пять) процентов цены контракта'
print(nlp(query)._.cats)


{'обеспечение исполнения контракта': 6.221935906648112e-06, '1': 0.9999937780640933}


In [93]:
result = {}
for sentence in df.loc[7, 'text']:
  result[sentence] = nlp(sentence)._.cats['обеспечение исполнения контракта']

In [91]:
pprint(result)

{'В случае заключения контракта по результатам определения поставщиков (подрядчиков, исполнителей) в соответствии с пунктом 1 части 1 статьи 30 закона № 44-ФЗ размер обеспечения исполнения контракта, в том числе предоставляемого с учетом положений статьи 37 закона № 44-ФЗ, устанавливается от цены, по которой заключается контракт, но не может составлять менее чем размер аванса ]': 2.5323663512785765e-06,
 'В случае установления требования размер обеспечения определяется в соответствии с ч  6 1 статьи 96 федерального закона «О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд»': 0.004318998454738165,
 'Идентификационный код закупки: 223631102474263110100100290020000244': 0.010063424848942747,
 'КОНТРАКТ №____ на поставку продуктов питания город ______________ «___»__________ 20__ год': 0.06871692253079328,
 'КОРПУС\'\', именуемое в дальнейшем «заказчик», в  который является казенным учреждением; 2) осуществления закупки услуги 

Видим, что чем меньше полученное значение, тем ближе наш текст по смыслу

Получили отличные результаты, осталось разделить на трейн/тест и посчитать значение accuracy